In [38]:
import pandas as pd 
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [26]:
train_df = pd.read_csv("./1b/train.csv").drop("Id", axis=1)
train_df

,y,x1,x2,x3,x4,x5
0,-6.822679,0.02,0.05,-0.09,-0.43,-0.08
1,-6.326290,-0.13,0.11,-0.08,-0.29,-0.03
2,-9.302728,0.08,0.06,-0.07,-0.41,-0.03
3,-7.371893,0.02,-0.12,0.01,-0.43,-0.02
4,-6.027647,-0.14,-0.12,-0.08,-0.02,-0.08
...,...,...,...,...,...,...
695,-6.147394,0.31,-0.09,0.04,-0.09,0.03
696,-6.214899,-0.26,-0.01,0.02,-0.40,0.05
697,-6.841341,-0.27,-0.22,-0.01,-0.32,-0.05
698,-6.523371,0.19,0.11,-0.05,-0.27,-0.04


In [27]:
X = train_df.iloc[:, 1:]
y = train_df['y']

In [28]:
#Feature transformations

# x^2
X[['square1','square2','square3','square4','square5']] = X.apply(np.square)
# exp(x)
X[['exp1','exp2','exp3','exp4','exp5']] = X.loc[:,'x1':'x5'].apply(np.exp)
# cos(x)
X[['cos1','cos2','cos3','cos4','cos5']] = X.loc[:,'x1':'x5'].apply(np.cos)

In [29]:
X

,x1,x2,x3,x4,x5,square1,square2,square3,square4,square5,exp1,exp2,exp3,exp4,exp5,cos1,cos2,cos3,cos4,cos5
0,0.02,0.05,-0.09,-0.43,-0.08,0.0004,0.0025,0.0081,0.1849,0.0064,1.020201,1.051271,0.913931,0.650509,0.923116,0.999800,0.998750,0.995953,0.908966,0.996802
1,-0.13,0.11,-0.08,-0.29,-0.03,0.0169,0.0121,0.0064,0.0841,0.0009,0.878095,1.116278,0.923116,0.748264,0.970446,0.991562,0.993956,0.996802,0.958244,0.999550
2,0.08,0.06,-0.07,-0.41,-0.03,0.0064,0.0036,0.0049,0.1681,0.0009,1.083287,1.061837,0.932394,0.663650,0.970446,0.996802,0.998201,0.997551,0.917121,0.999550
3,0.02,-0.12,0.01,-0.43,-0.02,0.0004,0.0144,0.0001,0.1849,0.0004,1.020201,0.886920,1.010050,0.650509,0.980199,0.999800,0.992809,0.999950,0.908966,0.999800
4,-0.14,-0.12,-0.08,-0.02,-0.08,0.0196,0.0144,0.0064,0.0004,0.0064,0.869358,0.886920,0.923116,0.980199,0.923116,0.990216,0.992809,0.996802,0.999800,0.996802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0.31,-0.09,0.04,-0.09,0.03,0.0961,0.0081,0.0016,0.0081,0.0009,1.363425,0.913931,1.040811,0.913931,1.030455,0.952334,0.995953,0.999200,0.995953,0.999550
696,-0.26,-0.01,0.02,-0.40,0.05,0.0676,0.0001,0.0004,0.1600,0.0025,0.771052,0.990050,1.020201,0.670320,1.051271,0.966390,0.999950,0.999800,0.921061,0.998750
697,-0.27,-0.22,-0.01,-0.32,-0.05,0.0729,0.0484,0.0001,0.1024,0.0025,0.763379,0.802519,0.990050,0.726149,0.951229,0.963771,0.975897,0.999950,0.949235,0.998750
698,0.19,0.11,-0.05,-0.27,-0.04,0.0361,0.0121,0.0025,0.0729,0.0016,1.209250,1.116278,0.951229,0.763379,0.960789,0.982004,0.993956,0.998750,0.963771,0.999200


In [39]:
model = LinearRegression()
model.fit(X,y)

LinearRegression()

In [45]:
print(model.intercept_)

-6689.189579816848


In [41]:
model.score(X,y)

0.03564105329279044

In [40]:
model.coef_

array([   87.19891672,   -67.27010868,    28.53822965,   158.07550392,
        -819.69642957,   200.04958807,  -393.54175642,  1953.26216011,
        -523.63288817,  1518.93422167,   -86.60565699,    65.34027498,
         -30.9921308 ,  -155.77167729,   822.10809668,   327.109988  ,
        -737.64020339,  3890.67031219, -1211.22831197,  3799.82175151])

In [44]:
ridge = make_pipeline(LinearRegression)
lambdas = {
    'ridge__alpha': [0.001, 0.1, 1, 10, 100, 200]
}
cross_validation_estimator = GridSearchCV(
                            ridge,
                            scoring='neg_mean_squared_error',
                            refit=False,
                            cv=10,
                            n_jobs=4,
                            param_grid=None,
                            return_train_score=True)
cross_validation_estimator.fit(X, y)

TypeError: 'NoneType' object is not iterable

In [36]:
rmse = np.sqrt(-cross_validation_estimator.cv_results_['mean_test_score'])
rmse

array([1.97788754, 1.97272118, 1.96727507, 1.96498025, 1.96816296,
       1.96892417])